In [77]:
import sys
import getpass
import os
import pandas as pd
import fiona
import geopandas as gpd
from arcgis import gis
username = getpass.getuser()

user = getpass.getuser()
sys.dont_write_bytecode = True

# for DataViz team members
sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *

In [80]:
client = gis.GIS(url='https://arcgis.ad.mtc.ca.gov/portal/home/', 
                 username=username, 
                 verify_cert=False)

Enter password: ········


In [81]:
transit_dir = os.path.join('/Users',
                        username,
                        'Box',
                        'DataViz Projects',
                        'Spatial Analysis and Mapping',
                        'Active Transportation Plan',
                        'Data',
                        'regional_transit')

In [82]:
active_trans_dir = os.path.join('/Users',
                        username,
                        'Box',
                        'DataViz Projects',
                        'Spatial Analysis and Mapping',
                        'Active Transportation Plan',
                        'Data',
                        'Active Transportation.gdb')

## Read most recent transit data from

In [83]:
transit_rts_precovid = pull_geodata_from_argis(arcgis_data_id='2a519083d0a44a33940e469e427c8457',
                                               client=client)

In [84]:
transit_rts_current = gpd.read_file(active_trans_dir,
                            driver='FileGDB',
                            layer='regional_transit_routes_oct_2021')


In [85]:
#Rename current columns to match precovid columns
rename_dict = {'route_short_name':'route_s_nm',
               'route_long_name':'route_l_nm',
               'route_type_text':'route_type'}
#Drop preexisting route_type which is numerical. We don't want to have duplicate column names
transit_rts_current = transit_rts_current.drop(['route_type'],axis=1)
transit_rts_current.rename(columns=rename_dict,inplace=True)

In [86]:
transit_rts_current.rename_geometry(col='geom',inplace=True)

In [87]:
#Add col to pre-covid routes to indicate vintage
transit_rts_precovid['vintage'] = 'January 2020'

In [88]:
#Add col to current routes to indicate vintage
transit_rts_current['vintage'] = 'October 2021'

In [89]:
#Drop Napa Valley Vine Transit pre-covid route E
transit_rts_precovid.drop(transit_rts_precovid[transit_rts_precovid['route_id'] == 'VN:E'].index,
                          inplace=True)

In [150]:
transit_rts_current[(transit_rts_current['agency_id'] == 'SC') & (transit_rts_current['route_id'].str.contains('568'))]

,Shape_Length,shape_id,route_id,agency_id,route_s_nm,route_l_nm,route_desc,route_url,route_color,route_text_color,route_sort_order,network_id,as_route,route_type,geom,vintage
1616,0.505220,SC:104378,SC:Rapid 568,SC,Rapid 568,Gilroy Transit Center to San Jose Diridon,Rapid buses may depart up to five minutes earl...,https://www.vta.org/go/routes/rapid-568,e4002b,FFFFFF,0,SC,0,Bus,"MULTILINESTRING ((-121.56651 37.00358, -121.56...",October 2021
1617,0.508949,SC:104379,SC:Rapid 568,SC,Rapid 568,Gilroy Transit Center to San Jose Diridon,Rapid buses may depart up to five minutes earl...,https://www.vta.org/go/routes/rapid-568,e4002b,FFFFFF,0,SC,0,Bus,"MULTILINESTRING ((-121.90223 37.33046, -121.90...",October 2021


In [151]:
#Subset new Napa Valley Vine Transit Routes and ACE Routes
out_cols = ['route_id',
            'agency_id',
            'route_s_nm',
            'route_l_nm',
            'vintage',
            'geom']
trans_rts_current_sub = transit_rts_current[out_cols][transit_rts_current['route_id']
                                                     .isin(['VN:E','VN:S','VN:N','VN:W','SC:Rapid 568','CE:ACE'])]

In [152]:
trans_rts_current_sub

,route_id,agency_id,route_s_nm,route_l_nm,vintage,geom
599,CE:ACE,CE,ACE,Altamont Corridor Express,October 2021,"MULTILINESTRING ((-121.27895 37.95706, -121.27..."
600,CE:ACE,CE,ACE,Altamont Corridor Express,October 2021,"MULTILINESTRING ((-121.90319 37.32957, -121.90..."
1616,SC:Rapid 568,SC,Rapid 568,Gilroy Transit Center to San Jose Diridon,October 2021,"MULTILINESTRING ((-121.56651 37.00358, -121.56..."
1617,SC:Rapid 568,SC,Rapid 568,Gilroy Transit Center to San Jose Diridon,October 2021,"MULTILINESTRING ((-121.90223 37.33046, -121.90..."
2970,VN:N,VN,N,Trancas | North Napa,October 2021,"MULTILINESTRING ((-122.28026 38.29754, -122.28..."
2971,VN:N,VN,N,Trancas | North Napa,October 2021,"MULTILINESTRING ((-122.31050 38.32224, -122.31..."
2972,VN:S,VN,S,Imola | South Napa,October 2021,"MULTILINESTRING ((-122.27984 38.29752, -122.27..."
2973,VN:S,VN,S,Imola | South Napa,October 2021,"MULTILINESTRING ((-122.30419 38.28765, -122.30..."
2975,VN:W,VN,W,West,October 2021,"MULTILINESTRING ((-122.28000 38.29751, -122.27..."
2976,VN:E,VN,E,East,October 2021,"MULTILINESTRING ((-122.27999 38.29751, -122.27..."


In [155]:
transit_rts_precov_new = pd.concat([transit_rts_precovid[out_cols],trans_rts_current_sub[out_cols]],
                                  axis=0,
                                  ignore_index=True,
                                  sort=False)

In [156]:
transit_rts_precov_new[transit_rts_precov_new['route_id']
                       .isin(['VN:E','VN:S','VN:N','VN:W','SC:Rapid 568','CE:ACE'])]

,route_id,agency_id,route_s_nm,route_l_nm,vintage,geom
5927,CE:ACE,CE,ACE,Altamont Corridor Express,October 2021,"MULTILINESTRING ((-121.27895 37.95706, -121.27..."
5928,CE:ACE,CE,ACE,Altamont Corridor Express,October 2021,"MULTILINESTRING ((-121.90319 37.32957, -121.90..."
5929,SC:Rapid 568,SC,Rapid 568,Gilroy Transit Center to San Jose Diridon,October 2021,"MULTILINESTRING ((-121.56651 37.00358, -121.56..."
5930,SC:Rapid 568,SC,Rapid 568,Gilroy Transit Center to San Jose Diridon,October 2021,"MULTILINESTRING ((-121.90223 37.33046, -121.90..."
5931,VN:N,VN,N,Trancas | North Napa,October 2021,"MULTILINESTRING ((-122.28026 38.29754, -122.28..."
5932,VN:N,VN,N,Trancas | North Napa,October 2021,"MULTILINESTRING ((-122.31050 38.32224, -122.31..."
5933,VN:S,VN,S,Imola | South Napa,October 2021,"MULTILINESTRING ((-122.27984 38.29752, -122.27..."
5934,VN:S,VN,S,Imola | South Napa,October 2021,"MULTILINESTRING ((-122.30419 38.28765, -122.30..."
5935,VN:W,VN,W,West,October 2021,"MULTILINESTRING ((-122.28000 38.29751, -122.27..."
5936,VN:E,VN,E,East,October 2021,"MULTILINESTRING ((-122.27999 38.29751, -122.27..."


## Read transit routes classification xlsx

In [162]:
transit_class = pd.read_excel(os.path.join(transit_dir,'BA_Agency_Category_0821.xlsx'),
                              sheet_name=2)

In [163]:
#Fix BART and ACE route ids to match shape ids
ba_dict = {'ACE':'CE:ACE',
    'BA:BG-N': 'BA:Beige',
 'BA:BG-S': 'BA:Beige',
 'BA:BL-N': 'BA:Blue-Wkd/Sat',
 'BA:BL-S': 'BA:Blue-Wkd/Sat',
 'BA:GN-N': 'BA:Green',
 'BA:GN-S': 'BA:Green',
 'BA:OR-N': 'BA:Orange',
 'BA:OR-S': 'BA:Orange',
 'BA:PR-N': 'BA:Purple',
 'BA:PR-S': 'BA:Purple',
 'BA:RD-N': 'BA:Red',
 'BA:RD-S': 'BA:Red',
 'BA:YL-N': 'BA:Yellow',
 'BA:YL-S': 'BA:Yellow'}

transit_class['route_id'].replace(ba_dict,inplace=True)

In [164]:
transit_class[transit_class['route_id'] == 'CE:ACE']

,route_id,agency_id,agency,route_short_name,route_long_name,category,sub_category,avg_peak_freq (min),avg_offpeak_freq (min),notes/comments
155,CE:ACE,CE,Altamont Corridor Express,ACE,Altamont Corridor Express,NaN,NaN,NaN,NaN,NaN


In [165]:
transit_rts_class = pd.merge(transit_rts_precov_new[['route_id','geom']],
                             transit_class,
                             how='inner',
                             on='route_id')

In [166]:
transit_rts_class[transit_rts_class['route_id'] == 'SC:Rapid 568']

,route_id,geom,agency_id,agency,route_short_name,route_long_name,category,sub_category,avg_peak_freq (min),avg_offpeak_freq (min),notes/comments
5773,SC:Rapid 568,"MULTILINESTRING ((-121.56651 37.00358, -121.56...",SC,VTA,Rapid 568,NaN,Regional,Regional,30,30,NaN
5774,SC:Rapid 568,"MULTILINESTRING ((-121.90223 37.33046, -121.90...",SC,VTA,Rapid 568,NaN,Regional,Regional,30,30,NaN


## Write non-match files to csv

In [167]:
# transit_rts_class[transit_rts_class['_merge'] == 'right_only']['agency_id'].unique()

In [168]:
# transit_rts_class[transit_rts_class['agency_id'] == 'CE']

In [169]:
# transit_rts_class[transit_rts_class['_merge'] == 'both']

In [170]:
# class_out_cols = ['agency_id',
#                   'agency',
#                   'route_short_name',
#                   'route_long_name',
#                   'category',
#                   'sub_category',
#                   'avg_peak_freq (min)',
#                   'avg_offpeak_freq (min)',
#                   'notes/comments']
# class_export_df = transit_rts_class[class_out_cols][transit_rts_class['_merge'] == 'right_only'].copy()

In [171]:
# agency_ids = ['CT', 'CC', 'SM', 'SF', 'VN', 'SC']
# shape_out_cols = ['agency_id',
#                   'agency_nm',
#                   'route_id',
#                   'route_l_nm',
#                   'route_s_nm',
#                   'route_type']
# shape_export_df = transit_rts_precovid[shape_out_cols][transit_rts_precovid['agency_id'].isin(agency_ids)].copy()

In [172]:
# with pd.ExcelWriter(os.path.join(transit_dir,'nonmatching_route_ids.xlsx')) as writer:
#     class_export_df.to_excel(writer, sheet_name='Unmatched_Route_Ids',index=False)
#     shape_export_df.to_excel(writer, sheet_name='Potential_Route_Id_Matches',index=False)

In [173]:
transit_rts_class.head(5)

,route_id,geom,agency_id,agency,route_short_name,route_long_name,category,sub_category,avg_peak_freq (min),avg_offpeak_freq (min),notes/comments
0,SR:4,"MULTILINESTRING ((-122.71373 38.43869, -122.71...",SR,Santa Rosa CityBus,4,Rincon Valley,Local,Local,60,60,NaN
1,SR:4,"MULTILINESTRING ((-122.71373 38.43869, -122.71...",SR,Santa Rosa CityBus,4,Rincon Valley,Local,Local,60,60,NaN
2,SR:4,"MULTILINESTRING ((-122.71373 38.43869, -122.71...",SR,Santa Rosa CityBus,4,Rincon Valley,Local,Local,60,60,NaN
3,SR:4,"MULTILINESTRING ((-122.71373 38.43869, -122.71...",SR,Santa Rosa CityBus,4,Rincon Valley,Local,Local,60,60,NaN
4,SR:4,"MULTILINESTRING ((-122.71373 38.43869, -122.71...",SR,Santa Rosa CityBus,4,Rincon Valley,Local,Local,60,60,NaN


In [174]:
#Export matching records to geojson
(transit_rts_class
 .to_file(os.path.join(transit_dir,'regional_transit_routes.geojson'),driver='GeoJSON'))